# --- Day 12: Christmas Tree Farm ---

In [1]:
# %%capture
!python -m pip install -q tqdm ipywidgets


# --- Support Functions ---
# This section contains support functions used by the main code.

from typing import List, Tuple, Dict, Set
from tqdm.notebook import tqdm
import time

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()
    
def parse_shapes_and_regions(input: str) -> Tuple[List[List[str]], List[Tuple[int, int, List[int]]]]:
    lines = [l.rstrip() for l in input]
    shapes: Dict[int, List[str]] = {}
    regions: List[Tuple[int,int,List[int]]] = []
    i = 0
    # Parse shapes
    while i < len(lines):
        line = lines[i]
        if not line:
            i += 1
            continue
        if ':' in line and line.split(':')[0].isdigit():
            idx = int(line.split(':')[0])
            i += 1
            grid = []
            while i < len(lines) and lines[i] and not (('x in lines[i]') and (':' in lines[i])):
                grid.append(lines[i])
                i += 1
            shapes[idx] = grid
        else:
            break
    # Parse regions
    while i < len(lines):
        line = lines[i]
        if not line:
            i += 1
            continue
        left, right = line.split(':')
        w, h = int(left.split('x')[0]), int(left.split('x')[1])
        counts = [int(x) for x in right.strip().split()] if right.strip() else []
        regions.append((w, h, counts))
        i += 1
    ordered_shapes = [shapes[i] for i in range(max(shapes.keys())+1)]
    return ordered_shapes, regions

In [2]:
# Specific Solution Functions

def grid_to_points(grid: List[str]) -> Set[Tuple[int,int]]:
    """ 
    Tranform the shape from a list of string where '#' is part of shape
    in a set of tuples containing the coordinate of each shape part
    """
    return {(x,y) for y, row in enumerate(grid) for x,ch in enumerate(row) if ch == '#'}


def transform_points(pts: Set[Tuple[int,int]], op: str)-> Set:
    """
    Rotate or Flip/rotate the shape.
    The operation is a string composed by operation and a number [0 to 3]
    to indicate the rotation. If the string starts with 'F' means a flipping.
    Possible strings are: rot0, rot1, rot2, rot3, Frot0, Frot1, Frot2, Frot3
    """
    mirror = op.startswith('F')
    r = int(op[-1])
    def rot(p):
        # rotate around (0,0)"
        x, y = p
        if r == 0: return (x, y)
        if r == 1: return (-y, x)
        if r == 2: return (-x, -y)
        if r == 3: return (y, -x)
    def normalize(points):
        # move shape to only positive coordinates starting from 0,0
        minx = min(x for x, _ in points)
        miny = min(y for _, y in points)
        return tuple(sorted(((x - minx, y - miny) for x, y in points)))
    pts2 = pts
    if mirror:
        pts2 = {(-x, y) for x,y in pts2}
    return normalize({rot(p) for p in pts2})

def unique_orientations(pts: Set[Tuple[int,int]])->List:
    variants = set()
    for op in ['rot0','rot1','rot2','rot3','Frot0','Frot1','Frot2','Frot3']:
        variants.add(transform_points(pts, op))
    return sorted(list(variants))

def shape_bounds(norm_pts):
    maxx = max (x for x, _ in norm_pts)
    maxy = max (y for _, y in norm_pts)
    return (maxx+1, maxy+1)

def build_placements(
        W: int, H: int, shapes_grid: List[List[str]]
    ) -> Tuple[ List, List]:
    """
    Prepare a list of all possible position and rotations of the shapes inside the region W x H.
    It receive a list of shapes (shapes_grid) and for each shape i returns a list of 
    all possible position expressed in a Tuple with:
    - Tuple with cell_index occupied by the shape
        index is created by counting all posizions starting from 0,0 (index 0) to W-1,H-1 index (W*H)-1
        so for example for W, H = 4,4  a index of 6 means position 2,1  (index = W*y + x)
    - Tuple with (x, y) position of the shape
    - Tuple of Tuple with the normalized coordinates of the shape components

    returns:
    - the placements: all possible placements 
    - areas : a list of areas occupied by shapes  area[0] is the area of shape[0]
                area is the count of elements composing the area 
    """
    shapes_pts = [grid_to_points(g) for g in shapes_grid]
    shapes_orients = [unique_orientations(pts) for pts in shapes_pts]
    placements_by_shape : List[List[Tuple[Set[int], Tuple[int,int], Tuple[Tuple[int,int], ...]]]] = []
    cell_index = { (x,y): y*W + x for y in range(H) for x in range(W)}
    for orients in shapes_orients:
        shape_list = []
        for norm in orients:
            bw, bh = shape_bounds(norm)
            for oy in range(H -bh +1): # all possible positions in y
                for ox in range(W - bw +1):
                    cells = {cell_index[(ox+dx, oy+dy)] for (dx, dy) in norm}
                    shape_list.append((cells, (ox, oy), norm))
        placements_by_shape.append(shape_list)

    def shape_area(pts): return len(pts)
    areas = [shape_area(orients[0]) for orients in shapes_orients]    
    return placements_by_shape, areas

def can_fit_region(
        W: int, H: int, counts: List[int], 
        shapes_grid: List[List[str]], need_solution: bool=False
    ) -> Tuple[bool, List[str] | None]:
    # For each shape contruct all different placements
    placements_by_shape, areas = build_placements(W,H,shapes_grid)
    # based on area occupied by each shape and the number of shapes to be fitted
    # calculate the total_area needed
    total_area = sum(c*a for c,a in zip(counts, areas))
    if total_area > W*H:   # No space in region to fill all requested shapes
        return False, None
    pieces: List[int] = []   # will contain the list of all shapes to be fitted
    for si, cnt in enumerate(counts):
        pieces.extend([si]*cnt)
    if not pieces:
        return True, ['.'*W for _ in range(H)] if need_solution else None

    # Heuristic: try the most constrained pieces first
    pieces.sort(key=lambda si: len(placements_by_shape[si]))

    occupied: Set[int] = set()
    selection: List[Tuple[int, Tuple[Set[int], Tuple[int,int], Tuple[Tuple[int,int], ...]]]] = []

    def backtrack (idx:int) -> bool:
        if idx == len(pieces): return True
        si = pieces[idx]
        # Placements filtered by current occupancy
        candidates = [p for p in placements_by_shape[si] if not (p[0] & occupied)]
        # Optional: sort by centrality to find solutions faster
        for cells, tl, norm in candidates:
            selection.append((si, (cells, tl, norm)))
            occupied.update(cells)
            if backtrack(idx+1):
                return True
            for c in cells: occupied.remove(c)
            selection.pop()
        return False

    feasable = backtrack(0)
    if not feasable:
        return False, None

    if not need_solution:
        return True, None

    # Render a grid with letters A,B,C,... per shape index
    grid = [['.' for _ in range(W)] for _ in range(H)]
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for si, (cells, tl, norm) in selection:
        letter = letters[si]
        ox, oy = tl
        for dx, dy in norm:
            grid[oy+dy][ox+dx] = letter
    return True, [''.join(row) for row in grid]

def _print_shape(pts: Set[Tuple[int,int]]):   # DEBUG to check the shape
    xmax = max(x for x, _ in pts)+1
    ymax = max(y for _, y in pts)+1
    shape = ['.'*xmax]*ymax
    for x,y in pts:
        txt = shape[y]
        shape[y] = txt[:x]+'#'+txt[x+1:]
    for l in shape:
        print(l)

def _print_placement(W: int, H: int, placement):
    cells, (x, y), shape = placement
    for py in range(H):
        line = ''
        for px in range(W):
            cell_index = px + py*W
            line = line + ( '#' if cell_index in cells else '.')
        print(line)





In [3]:
# --- Part ONE ---

input = read_input('input.txt')

result = 0

shapes, regions = parse_shapes_and_regions(input)

# print (shapes)
# print (regions)

# W, H, counts = regions[0]

# pts = grid_to_points(shapes[0])
# print ("Shape[0] points:", pts)
# _print_shape(pts)
# orientations = unique_orientations(pts)
# print ("Orientations :")
# for o in orientations:
#     print("------------")
#     print_shape (o)
# print("------------")
# placements, areas = build_placements(W, H, shapes[:1])
# print ("placements =", placements)
# print ("areas =", areas)
# print ("region (",W,",",H,")")
# print ("------------")

# for placement in placements:
#     for shape_placement in placement:
#         _print_placement(W, H, shape_placement)
#         print ("------------")

fits = 0
for (W, H, counts) in tqdm(regions, desc="Processing regions"):
    ok, grid = can_fit_region(W, H, counts, shapes, need_solution=False)
    print(f"{W}x{H} {counts} -> {'FIT' if ok else 'NO FIT'}")
    if ok and grid:
        print("\n".join(grid))
        print()
    if ok: fits += 1

result = fits
    
print("Part ONE:", result)

Processing regions:   0%|          | 0/1000 [00:00<?, ?it/s]

37x43 [28, 31, 34, 28, 24, 23] -> FIT
48x41 [26, 34, 37, 40, 39, 31] -> FIT
38x39 [37, 36, 41, 43, 35, 37] -> NO FIT
46x39 [37, 44, 48, 38, 62, 48] -> NO FIT
50x35 [29, 29, 33, 26, 24, 34] -> FIT
46x47 [26, 42, 38, 46, 33, 39] -> FIT
49x35 [48, 38, 47, 38, 44, 51] -> NO FIT
49x43 [41, 41, 33, 28, 42, 39] -> FIT
50x38 [35, 33, 24, 34, 36, 30] -> FIT
46x49 [60, 47, 68, 55, 55, 66] -> NO FIT
40x49 [48, 45, 45, 64, 55, 42] -> NO FIT
50x37 [34, 34, 36, 30, 27, 30] -> FIT
47x48 [49, 41, 37, 43, 36, 33] -> FIT
36x40 [24, 21, 28, 30, 26, 27] -> FIT
36x41 [40, 39, 34, 32, 40, 42] -> NO FIT
43x48 [44, 38, 40, 23, 51, 27] -> FIT
35x50 [31, 30, 33, 33, 20, 29] -> FIT
41x47 [35, 31, 31, 32, 32, 33] -> FIT
46x48 [48, 37, 40, 41, 36, 37] -> FIT
44x39 [45, 46, 44, 47, 39, 43] -> NO FIT
37x45 [37, 40, 44, 43, 42, 52] -> NO FIT
36x50 [48, 42, 47, 45, 40, 57] -> NO FIT
49x38 [24, 39, 33, 25, 37, 33] -> FIT
38x37 [37, 23, 50, 35, 43, 32] -> NO FIT
48x43 [34, 39, 41, 33, 30, 47] -> FIT
50x44 [49, 63, 55, 4